In [27]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import re
import numpy as np
import os
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [28]:
###importing the data and checking through for anormalies 
data2021 = pd.read_csv('2021b table uncleaned -Copy1.csv', parse_dates=['date'])
data2021.tail(50)
data2021.isnull().sum()
data2021.shape[0]

175792

HERE BECAUSE OF THE FILEPATH AND DIMENSIONM BOTHE CARRIED USEFUL INFORMTION; I INTUITIVELY ADDED BOTH CIOLUMN TOGETHER IN ORDER NOT TO MISS ANY DATA 

In [4]:
##creating a check point and droping the first rows##
####
data2021.columns
##the addition below is very important in other to avioid loosing data from the other colums with nan
data2021['dimension']= data2021['pagePath'].astype(str) + data2021['dimension36'].astype(str)### 

In [5]:
relevant = ['viewid','date','pagePath','hits','dimension']
data_to_clean = data2021[relevant]
data_to_clean.isnull().sum()

viewid       0
date         0
pagePath     0
hits         0
dimension    0
dtype: int64

In [10]:
##replacing url column over with data from pagepath 
#data2019.dimension36.fillna(data2019.pagePath, inplace=True) ##replacing data in the columns (dimension url with pagePath data)

##run check##
#data2020.isnull().sum()

CREATING CODE TO EXTRACT THE NEWNESS TYPE 

In [11]:
##creating function for to find out whether car is new or new or old (Newness Type) in pagePath# ##
data_to_clean["newness_type"]=data_to_clean["dimension"].str.extract('(new|used)')
data_to_clean['newness_type'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type'].unique() ##check unique values 
data_to_clean['newness_type'].value_counts() ##count unique values 

unknown_type    149171
used             21259
new               5362
Name: newness_type, dtype: int64

In [12]:
##creating function for to find out whether car is new or new or old in Dimension36 (As Newness Type) ##
data_to_clean["newness_type2"]=data_to_clean["dimension"].str.extract('(new|used)')
data_to_clean['newness_type2'].fillna('unknown_type', inplace=True)
data_to_clean['newness_type2'].unique() ##check unique values 
data_to_clean['newness_type2'].value_counts() ##count unique values 

unknown_type    149171
used             21259
new               5362
Name: newness_type2, dtype: int64

CHECKING FOR THE NEWNESS TYPE 

In [14]:
###viewing data without unknown newness type##
data_to_clean[data_to_clean['newness_type']== 'unknown_type'].head(5)

,viewid,date,pagePath,hits,dimension,newness_type2,newness_type
0,129875374,2021-03-10,/acura/ilx/,1,/acura/ilx/nan,unknown_type,unknown_type
1,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type,unknown_type
2,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type,unknown_type
3,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type,unknown_type
4,129875374,2021-03-10,/acura/mdx/,4,/acura/mdx/nan,unknown_type,unknown_type


CHECKING HOW MANY VALUES WERE EXTRACTED 

In [15]:
data_to_clean['newness_type'].unique()
data_to_clean['newness_type'].value_counts()

unknown_type    149171
used             21259
new               5362
Name: newness_type, dtype: int64

In [16]:
check = ["newness_type2","newness_type"]
data_to_clean[check]
#data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,newness_type2,newness_type
0,unknown_type,unknown_type
1,unknown_type,unknown_type
2,unknown_type,unknown_type
3,unknown_type,unknown_type
4,unknown_type,unknown_type
...,...,...
175787,unknown_type,unknown_type
175788,unknown_type,unknown_type
175789,unknown_type,unknown_type
175790,unknown_type,unknown_type


CONFIRMING THAT THE CODE RAN PROPERLY AND THERE IS NO MISSING INFORMATION WHERE THERE WAS A NEWNESS TYPE

In [17]:
data_to_clean[data_to_clean["newness_type2"]!= data_to_clean["newness_type"]]

,viewid,date,pagePath,hits,dimension,newness_type2,newness_type


In [19]:
### merging btoh newness column to have one column## 
data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2


##Drop  Newness_type2##
##clean_1 = data_to_clean[data_to_clean.newness_type!= 'unknown_type']
clean_1 = data_to_clean.drop(['newness_type2'], axis=1)##break point after removing newness_type2 data  or newness or use type

###clean_1.drop(['newness_type2'], axis=1, inplace=True) ##code if if I have to remove the unknown newness type 


clean_1.shape[0]

<ipython-input-19-f23abb027719>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_to_clean.newness_type[data_to_clean.newness_type=='unknown_type']=data_to_clean.newness_type2


175792

In [20]:
data_to_clean['newness_type'].value_counts()

unknown_type    149171
used             21259
new               5362
Name: newness_type, dtype: int64

In [21]:
##data_to_clean[data_to_clean['newness_type']== 'unknown_type'].tail(100)

In [22]:
clean_1.head() ##previewing data 


,viewid,date,pagePath,hits,dimension,newness_type
0,129875374,2021-03-10,/acura/ilx/,1,/acura/ilx/nan,unknown_type
1,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type
2,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type
3,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type
4,129875374,2021-03-10,/acura/mdx/,4,/acura/mdx/nan,unknown_type


In [16]:
#####working on the url (dimension36) too extract importanst data#
###first removing the url beginning ###
##removing the beginning url for consistencies in the column 


CREATING A CHECKPOINT

In [25]:
clean_2= clean_1.copy()
clean_2[0:10]

,viewid,date,pagePath,hits,dimension,newness_type
0,129875374,2021-03-10,/acura/ilx/,1,/acura/ilx/nan,unknown_type
1,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type
2,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type
3,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type
4,129875374,2021-03-10,/acura/mdx/,4,/acura/mdx/nan,unknown_type
5,129875374,2021-03-10,/acura/mdx/2012/vin/2hnyd2h25ch548630/,3,/acura/mdx/2012/vin/2hnyd2h25ch548630/nan,unknown_type
6,129875374,2021-03-10,/acura/mdx/2014/,1,/acura/mdx/2014/nan,unknown_type
7,129875374,2021-03-10,/acura/mdx/2016/vin/5fryd3h94gb019527/,3,/acura/mdx/2016/vin/5fryd3h94gb019527/nan,unknown_type
8,129875374,2021-03-10,/acura/mdx/2016/vin/5fryd4h44gb025161/?radius=50,1,/acura/mdx/2016/vin/5fryd4h44gb025161/?radius=...,unknown_type
9,129875374,2021-03-10,/acura/mdx/2016/vin/5fryd4h4xgb029148/,1,/acura/mdx/2016/vin/5fryd4h4xgb029148/nan,unknown_type


EXTRACTING THE DATA (MAKE, MODEL, YEAR, INTERIOR COLOR, EXTERIOR COLOR)
STEP 1: CREATING ANOTHER COLUMN CALLED URL BY INTRODUCING A MOLE(WHICH IS THE SPACE             BAR('')) TO EXTRACT DATA FROM ADDRESS THAT ENDED WITHOUT AN NONALPHANUMERIC OR         EMPTY STRING
STEP 2: CREATING THE REGEX MODEL TO CAPTURE DATA THAT HAVE THE SPECIAL CHARACTERS                (=,%,&,AND THE SPACEBAR '' THAT I INTRODUCED ETC)  
STEP 3: REPLACING THE SEPARATORS TO (&) FOR INFORMITY AND EASY EXTRACTION
STEP 4: CHANGING ALL NULL VALUES TO UNKNOWN 
STEP 5: THE ACTUAL EXTRACTION USING THE SPECIAL CHARACTER AS THE BREAK POINT
STEP 6: ERASING THE LEFT OVER CHARACTER (ow,(OEM/OEMB) or w(MODEL/MODELB) or o(YEAR)
STEP 7: MERGE EXTRACTION FROM THE DUPLICAATE COLUMS INTO ONE USING THE (UNKWON                  PARAMETER)
STEP 8: CREATING A DATAFRAME FOR COLOR TO BE REJOINED LATER 
STEP 9: DROPPING UNWANTED COLUMNS (MODELB,OEMB)
    
    I TOOK CARE OF THE FOLLOWING TYPES 
    https://statefarm.edmunds.com/srp/?make=Toyota%26model=Sequoia%26type=used
    https://www.edmunds.com/Ford/F-150/2012/vin/1FTFW1ET5CFB39172/utm_source=cordial&utm_medium=email&utm_account=edmunds_insider&utm_campaign=member_alert&utm_term=price_drop_alert
    /lexus/ls-430/2005/vin/jthbn36f855015006/

In [18]:
####Extracting Values for make, model and year

#STEP 1
###first create another variable "url" by modifying "variable dimension36" which will have a spacebar the end for uniformity ##
clean_2['url'] = clean_2['dimension'].astype(str) + ' '

##STEP 2
####Extracting Values for make, model and year
clean_2['Maker']=clean_2['dimension'].str.extract('(make=[\w+-]+\W)')
clean_2['Makerb']=clean_2['url'].str.extract('(make=[a-zA-Z0-9_\%\W]+ $)')
clean_2['Model']=clean_2['dimension'].str.extract('(model=[\w+-]+\W)')
clean_2['Modelb']=clean_2['url'].str.extract('(model=[a-zA-Z0-9_\%\W]+ $)')
clean_2['Year']=clean_2['dimension'].str.extract('(year=......)')
clean_2['Extcolor']=clean_2['url'].str.extract('(extcolor=[\w+]+\W)')
clean_2['Intcolor']=clean_2['url'].str.extract('(intcolor=[\w+]+\W)')
####clean_2['ModelB']=clean_2['url'].str.extract('(model=[a-zA-Z0-9_\-]+ $)')

#STEP 3
##replace the separator % with & for uniformity##
clean_2['Maker']=clean_2['Maker'].str.replace('%','&')
clean_2['Model']=clean_2['Model'].str.replace('%','&')
clean_2['Year']=clean_2['Year'].str.replace('%','&')
#clean_2['Modelb']=clean_2['Modelb'].str.replace(' ','&')###not necessary because we are splitting on spacebar
#clean_2['Makerb']=clean_2['Makerb'].str.replace(' ','&')###not necessary because we are splitting on spacebar


#STEP 4
##Fill all Null values within columns make, model and year####
clean_2['Maker'].fillna('unknown', inplace=True)
clean_2['Makerb'].fillna('unknown', inplace=True)
clean_2['Model'].fillna('unknown', inplace=True)
clean_2['Modelb'].fillna('unknown', inplace=True)
clean_2['Year'].fillna('unknown', inplace=True)
clean_2['Extcolor'].fillna('unknown', inplace=True)
clean_2['Intcolor'].fillna('unknown', inplace=True)


#STEP 5
###creating a cleaned column for variable make, model and yea## 
clean_2['OEM']=clean_2['Maker'].apply(lambda st: st[st.find("make=")+5:st.find("\W")])
clean_2['OEMB']=clean_2['Makerb'].apply(lambda st: st[st.find("make=")+5:st.find(" ")])
clean_2['MODEL']=clean_2['Model'].apply(lambda st: st[st.find("model=")+6:st.find("\W")])
clean_2['MODELB']=clean_2['Modelb'].apply(lambda st: st[st.find("model=")+6:st.find(" ")])
clean_2['YEAR_BUILT']=clean_2['Year'].str.extract('(\d+)') 
##clean_2['YEAR_BUILT']=clean_2['Year'].apply(lambda st: st[st.find("year=")+5:st.find("\W")-1])
clean_2['INTCOLOR']=clean_2['Extcolor'].apply(lambda st: st[st.find("extcolor=")+9:st.find("\W")])  ######this will dates without ranges 
clean_2['EXTCOLOR']=clean_2['Intcolor'].apply(lambda st: st[st.find("intcolor=")+9:st.find("\W")]) ###this will get data for dates that have ranges ranges


#STEP 6 
##replace ow/w values within columns for clarification####
clean_2['OEM'].replace({"ow": "unknown"}, inplace=True)
clean_2['OEMB'].replace({"ow": "unknown"}, inplace=True)
clean_2['MODEL'].replace({"w": "unknown"}, inplace=True)
clean_2['MODELB'].replace({"w": "unknown"}, inplace=True)
clean_2['YEAR_BUILT'].replace({"o": "unknown"}, inplace=True)

clean_2['INTCOLOR'] = clean_2.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
clean_2['EXTCOLOR'] = clean_2.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
#clean_2['INTCOLOR'].replace({" ": "unknown"}, inplace=True)
clean_2.INTCOLOR.fillna("unknown", inplace=True)
clean_2['EXTCOLOR'].fillna("unknown", inplace=True)



#STEP 7
###Merging both OEM and MODEL
clean_2.OEM[clean_2.OEM=='unknown']=clean_2.OEMB
clean_2.MODEL[clean_2.MODEL=='unknown']=clean_2.MODELB


##Drop unwanted columns##
####clean_2.drop(['Maker','Makerb','Model','Modelb','Year','MODELB','pagePath','url', 'OEMB'], axis=1, inplace=True)

##STEP 8
###Needed variables 
#var= ['viewid', 'date', 'hits','dimension36','newness_type','OEM','MODEL','YEAR_BUILT']
var= ['viewid', 'date', 'hits','pagePath','dimension','newness_type','OEM','MODEL','YEAR_BUILT']
color = ['EXTCOLOR','INTCOLOR']
Data_cleaned_part1 = clean_2[var]
color_df = clean_2[color]


<ipython-input-18-87d730ec4da8>:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_2.OEM[clean_2.OEM=='unknown']=clean_2.OEMB
<ipython-input-18-87d730ec4da8>:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_2.MODEL[clean_2.MODEL=='unknown']=clean_2.MODELB


In [19]:
Data_cleaned_part1

,viewid,date,hits,pagePath,dimension,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2021-03-10,1,/acura/ilx/,/acura/ilx/nan,unknown_type,unknown,unknown,NaN
1,129875374,2021-03-10,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type,unknown,unknown,NaN
2,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type,unknown,unknown,NaN
3,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type,unknown,unknown,NaN
4,129875374,2021-03-10,4,/acura/mdx/,/acura/mdx/nan,unknown_type,unknown,unknown,NaN
...,...,...,...,...,...,...,...,...,...
175787,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,unknown_type,unknown,unknown,NaN
175788,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius=25,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius...,unknown_type,unknown,unknown,NaN
175789,129875374,2021-03-09,1,/volvo/xc90/,/volvo/xc90/nan,unknown_type,unknown,unknown,NaN
175790,129875374,2021-03-09,1,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius=50,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius...,unknown_type,unknown,unknown,NaN


In [20]:
#clean_2['Modelb'].isnull().sum()
#clean_2[clean_2['Year']=='unknown']


In [21]:
clean_2['INTCOLOR'].value_counts()

unknown      174108
white           358
black           310
dark            236
gray            147
whitenan        146
blacknan        109
silver           80
red              54
light            53
graynan          52
off              44
silvernan        35
rednan           26
yellow            8
yellownan         7
orangenan         7
purple            4
purplenan         4
orange            4
Name: INTCOLOR, dtype: int64

In [22]:
###checking if theres  is color info###
clean_2[clean_2['INTCOLOR']!='']
#clean_2[8535:8550]

,viewid,date,pagePath,hits,dimension,newness_type,url,Maker,Makerb,Model,...,Year,Extcolor,Intcolor,OEM,OEMB,MODEL,MODELB,YEAR_BUILT,INTCOLOR,EXTCOLOR
0,129875374,2021-03-10,/acura/ilx/,1,/acura/ilx/nan,unknown_type,/acura/ilx/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
1,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
2,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
3,129875374,2021-03-10,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
4,129875374,2021-03-10,/acura/mdx/,4,/acura/mdx/nan,unknown_type,/acura/mdx/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175787,129875374,2021-03-09,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,1,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,unknown_type,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
175788,129875374,2021-03-09,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius=25,1,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius...,unknown_type,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
175789,129875374,2021-03-09,/volvo/xc90/,1,/volvo/xc90/nan,unknown_type,/volvo/xc90/nan,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown
175790,129875374,2021-03-09,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius=50,1,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius...,unknown_type,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius...,unknown,unknown,unknown,...,unknown,unknown,unknown,unknown,unknown,unknown,unknown,NaN,unknown,unknown


In [23]:
###checking if theres  is color info###
clean_2[clean_2['MODEL']!='unknown']
#clean_2[8535:8550]

,viewid,date,pagePath,hits,dimension,newness_type,url,Maker,Makerb,Model,...,Year,Extcolor,Intcolor,OEM,OEMB,MODEL,MODELB,YEAR_BUILT,INTCOLOR,EXTCOLOR
21,129875374,2021-03-10,/acura/rdx/build/?submodel=suv&modelyearid=401...,1,/acura/rdx/build/?submodel=suv&modelyearid=401...,unknown_type,/acura/rdx/build/?submodel=suv&modelyearid=401...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401846344&styleid=401857798&op...,NaN,unknown,unknown
22,129875374,2021-03-10,/acura/rdx/build/?submodel=suv&modelyearid=401...,1,/acura/rdx/build/?submodel=suv&modelyearid=401...,unknown_type,/acura/rdx/build/?submodel=suv&modelyearid=401...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401846344&styleid=401857800&op...,NaN,unknown,unknown
88,129875374,2021-03-10,/audi/q7/build/?submodel=suv&modelyearid=40187...,1,/audi/q7/build/?submodel=suv&modelyearid=40187...,unknown_type,/audi/q7/build/?submodel=suv&modelyearid=40187...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401873311&styleid=401873314&op...,NaN,unknown,unknown
124,129875374,2021-03-10,/bmw/5-series/build/?submodel=sedan&modelyeari...,1,/bmw/5-series/build/?submodel=sedan&modelyeari...,unknown_type,/bmw/5-series/build/?submodel=sedan&modelyeari...,unknown,unknown,model=sedan&,...,unknown,unknown,unknown,unknown,unknown,sedan,sedan&modelyearid=401852262&styleid=401874662&...,NaN,unknown,unknown
144,129875374,2021-03-10,/bmw/x3/build/?submodel=hybrid&modelyearid=401...,1,/bmw/x3/build/?submodel=hybrid&modelyearid=401...,unknown_type,/bmw/x3/build/?submodel=hybrid&modelyearid=401...,unknown,unknown,model=hybrid&,...,unknown,unknown,unknown,unknown,unknown,hybrid,hybrid&modelyearid=401866210&styleid=401866227...,NaN,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175522,129875374,2021-03-09,/toyota/tacoma/build/?submodel=double cab&mode...,1,/toyota/tacoma/build/?submodel=double cab&mode...,unknown_type,/toyota/tacoma/build/?submodel=double cab&mode...,unknown,unknown,model=double,...,unknown,unknown,unknown,unknown,unknown,double,double,NaN,unknown,unknown
175523,129875374,2021-03-09,/toyota/tacoma/build/?submodel=double cab&mode...,1,/toyota/tacoma/build/?submodel=double cab&mode...,unknown_type,/toyota/tacoma/build/?submodel=double cab&mode...,unknown,unknown,model=double,...,unknown,unknown,unknown,unknown,unknown,double,double,NaN,unknown,unknown
175545,129875374,2021-03-09,/toyota/venza/build/?submodel=suv&modelyearid=...,1,/toyota/venza/build/?submodel=suv&modelyearid=...,unknown_type,/toyota/venza/build/?submodel=suv&modelyearid=...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401851567&styleid=401862736&op...,NaN,unknown,unknown
175753,129875374,2021-03-09,/volkswagen/atlas/build/?submodel=suv&modelyea...,1,/volkswagen/atlas/build/?submodel=suv&modelyea...,unknown_type,/volkswagen/atlas/build/?submodel=suv&modelyea...,unknown,unknown,model=suv&,...,unknown,unknown,unknown,unknown,unknown,suv,suv&modelyearid=401838632&styleid=401840627&op...,NaN,unknown,unknown


EXPORT POINT TO VIEW OUTPUT 

In [24]:
##checking for the correctness of output among variables with values 
Data_cleaned_part1.to_csv('Data_cleaned_part12019aaa.csv', encoding='utf-8')

CLEANING THE SECOND SEGMENT USING SPLIT

In [26]:
STEPS 
STEP 1: SPLITTING THE DATA INTO TWO BASED ON OEM (MAKER THAT HAS BEEN EXTRACTED)
STEP 2: REPLACING THE STARTING STRINGS FROM (WWW//..........edmund....) to (/)
STEP 3: REPLACING (-) AND OTHER SPECIAL CHARACTER USED FOR SEPERATION INTO (/)
STEP 4: SPLIT THE URL INTO DIFFERENT COLUMNS BY (/)
STEP 5: SELECTING THE NEEDED COLUMN 
STEP 6: JOINING THE TABLE BACK TOGETHE
STEP 7: FILL ALL NULL VALUES IF ANY WITH UNKNOWN TO BE DROPPED LATER 
STEP 7: TRANSFORMING THE TABLE 
STEP 8: EXPORT CSV

SyntaxError: invalid syntax (<ipython-input-26-467400e6fa20>, line 2)

In [25]:
dataA = Data_cleaned_part1[Data_cleaned_part1['OEM']!='unknown'] 
dataB = Data_cleaned_part1[Data_cleaned_part1['OEM']=='unknown']

In [26]:
dataB

,viewid,date,hits,pagePath,dimension,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2021-03-10,1,/acura/ilx/,/acura/ilx/nan,unknown_type,unknown,unknown,NaN
1,129875374,2021-03-10,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type,unknown,unknown,NaN
2,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type,unknown,unknown,NaN
3,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type,unknown,unknown,NaN
4,129875374,2021-03-10,4,/acura/mdx/,/acura/mdx/nan,unknown_type,unknown,unknown,NaN
...,...,...,...,...,...,...,...,...,...
175787,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,unknown_type,unknown,unknown,NaN
175788,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius=25,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius...,unknown_type,unknown,unknown,NaN
175789,129875374,2021-03-09,1,/volvo/xc90/,/volvo/xc90/nan,unknown_type,unknown,unknown,NaN
175790,129875374,2021-03-09,1,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius=50,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius...,unknown_type,unknown,unknown,NaN


In [27]:
##Creating checkpoint##
###CHeckpoint###
#dataA[1510:1531]
#dataA.to_csv('dataA2020vv.csv',encoding='utf-8')
#dataB.to_csv('dataB2020cc.csv',encoding='utf-8')

var = ['viewid','date','hits','pagePath','dimension','newness_type'] 
to_clean = dataB[var]
to_clean


,viewid,date,hits,pagePath,dimension,newness_type
0,129875374,2021-03-10,1,/acura/ilx/,/acura/ilx/nan,unknown_type
1,129875374,2021-03-10,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type
2,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type
3,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type
4,129875374,2021-03-10,4,/acura/mdx/,/acura/mdx/nan,unknown_type
...,...,...,...,...,...,...
175787,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,unknown_type
175788,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius=25,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius...,unknown_type
175789,129875374,2021-03-09,1,/volvo/xc90/,/volvo/xc90/nan,unknown_type
175790,129875374,2021-03-09,1,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius=50,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius...,unknown_type


In [28]:
###first removing the beginning url string if any## 
to_clean["dimension"].replace(regex=['(http+[\W+\w+]+.edmunds.com)'], value=[''], inplace = True) ##removing the beginning url for consistencies in the column 
to_clean["dimension"].replace(regex=['(/+used+-)'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 
#to_clean["dimension36"].replace(regex=['(\A/+0-9+/)'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 
to_clean["dimension"].replace(regex=['-'], value=['/'], inplace = True) ##removing the beginning url for consistencies in the column 


##splitting the url by (/)
url = to_clean["dimension"].str.split('/', expand=True)

##naming the split columns ##
url.rename(columns={0:"empty", 1:"OEM", 2:"MODEL", 3:"YEAR_BUILT", 4:"design", 5:"exitpage1", 6:"exitpage2"}, inplace=True)

##dropping unwanted columns 
neededcolumn = ['OEM','MODEL','YEAR_BUILT'] 
useful = url[neededcolumn]

/Users/mayorbaba/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [29]:
useful

,OEM,MODEL,YEAR_BUILT
0,acura,ilx,nan
1,acura,ilx,2018
2,acura,ilx,2018
3,acura,ilx,2018
4,acura,mdx,nan
...,...,...,...
175787,volvo,xc70,2013
175788,volvo,xc70,2013
175789,volvo,xc90,nan
175790,volvo,xc90,2017


In [30]:
##Joining the dataframe back to ##
cleaned = pd.concat([to_clean, useful], axis=1)

In [31]:

cleaned 

,viewid,date,hits,pagePath,dimension,newness_type,OEM,MODEL,YEAR_BUILT
0,129875374,2021-03-10,1,/acura/ilx/,/acura/ilx/nan,unknown_type,acura,ilx,nan
1,129875374,2021-03-10,3,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=25,/acura/ilx/2018/vin/19ude2f32ja005360/?radius=...,unknown_type,acura,ilx,2018
2,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=100,/acura/ilx/2018/vin/19ude2f40ja004071/?radius=...,unknown_type,acura,ilx,2018
3,129875374,2021-03-10,1,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=100,/acura/ilx/2018/vin/19ude2f71ja005210/?radius=...,unknown_type,acura,ilx,2018
4,129875374,2021-03-10,4,/acura/mdx/,/acura/mdx/nan,unknown_type,acura,mdx,nan
...,...,...,...,...,...,...,...,...,...
175787,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,/volvo/xc70/2013/vin/yv4940bz3d1151130/?radius...,unknown_type,volvo,xc70,2013
175788,129875374,2021-03-09,1,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius=25,/volvo/xc70/2013/vin/yv4940bz4d1166543/?radius...,unknown_type,volvo,xc70,2013
175789,129875374,2021-03-09,1,/volvo/xc90/,/volvo/xc90/nan,unknown_type,volvo,xc90,nan
175790,129875374,2021-03-09,1,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius=50,/volvo/xc90/2017/vin/yv4a22pl5h1183926/?radius...,unknown_type,volvo,xc90,2017


In [32]:
###checkpoint to view output in excel##
#cleaned.to_csv('cleaned.csv',encoding='utf-8')

In [33]:
##Fill all Null values within columns make, model and year####
cleaned['OEM'].fillna('unknown', inplace=True)
cleaned['MODEL'].fillna('unknown', inplace=True)
#cleaned['YEAR_BUILT'].fillna('unknown', inplace=True)

In [34]:
####Next is joining the data together as one ##
joined_clean = cleaned.append(dataA)

####Next is joining the data together as one ##
joined_clean= cleaned.append(dataA)


##joining the color dataframe from the original dataframe
data_with_color = pd.concat([joined_clean, color_df], axis=1)

##clean color again##
data_with_color['INTCOLOR'] = data_with_color.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
data_with_color['EXTCOLOR'] = data_with_color.INTCOLOR.replace(r'^\s*$', np.NaN, regex=True)
data_with_color.INTCOLOR.fillna("unknown", inplace=True)
data_with_color['EXTCOLOR'].fillna("unknown", inplace=True)
data_with_color.shape[0]

175792

In [35]:
##checkpoint to check joined data###
#data_with_color.to_csv('joined_cleaned2019bba.csv',encoding='utf-8')
data_with_color.shape[0]

175792

In [36]:
###FINAL CLEANING OF MODEL ALL COLUMNS 
###cleaning up the year column to have only numbers###
data_with_color['YEAR_BUILT'] = data_with_color['YEAR_BUILT'].str.replace(r'\D+', '')
#data_with_color['YEAR_BUILT'] .replace({'': "unknown"}, inplace=True)
#data_with_color['YEAR_BUILT'].value_counts()

###SELECTING THE FINAL DATA
final_2018 = data_with_color.drop(['viewid','pagePath'],  axis=1)
#complete= data_with_color[data_with_color.YEAR_BUILT.apply(lambda x: x.isnumeric())] 
complete = final_2018[(final_2018['YEAR_BUILT']>'1800') & (final_2018['YEAR_BUILT']<'2022')]
#df[(df["A"] >= 1) & (df["B"] < 5)]
complete.shape[0]










<ipython-input-36-f75e03c98380>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_with_color['YEAR_BUILT'] = data_with_color['YEAR_BUILT'].str.replace(r'\D+', '')


87156

By Checking on the error files that had no date, we can see that they are really not relevant and they only carry information of data such as:
1. lead form certificate 
2. Exiting Thank you page
3. Promotional pages (e.g hyundai test drive )
4. Dealer locator link 
all of the link do not contain data that are relevant for analysis and therefore they will be dropped .


In [37]:
#check point to check through in excel##
#final_2018 
complete.to_csv('21timi final_cleaned_.csv',encoding='utf-8')

In [38]:
data_with_color.to_csv('2021a final_cleaned_.csv',encoding='utf-8')